In [ ]:
from PIL import Image
img = Image.open('point_rgb_mirror.jpg').convert('RGB')
img = np.array(img)
print(img.shape)

(4096, 4096, 3)


In [26]:
import torch.utils.data as data
import numpy as np
from PIL import Image
import os
import torch
import torchvision.transforms as transforms

In [28]:
data_path = 'origin'
img_paths = [os.path.join(data_path, filename) for filename in os.listdir(data_path)
                          if filename.endswith('line.png')]



In [29]:
for img_path in img_paths:
    line_img = Image.open(img_path).convert('L')
    point_img = Image.open(img_path.replace('line.png', 'point.png')).convert('L')
    speed = np.load(img_path.replace('line.png', 'speed.npy'))
    direction = np.load(img_path.replace('line.png', 'direction.npy'))
    transition_view = np.load(img_path.replace('line.png', 'transition.npy')).astype('float32')

    transition_view[transition_view > 0] = 1.0
    transition_view = torch.from_numpy(transition_view)
    #print('transition_view_before:',transition_view.shape)

    transition_view = transition_view.reshape([256, 256, 128]).permute(2, 0, 1)
    #print('transition_view_after:',transition_view.shape)

    # normalization & to torch data structure
    spatial_features = []
    img_transform = transforms.ToTensor()
    spatial_features.append(img_transform(point_img))
    line_img = img_transform(line_img)
    spatial_features.append(line_img)
    speed = torch.from_numpy((speed / 13.1).astype('float32'))
    spatial_features.append(speed.permute(2, 0, 1))  # final positions
    direction = direction.astype('float32')
    summed = np.sum(direction, axis=2, keepdims=True)
    direction = torch.from_numpy(np.divide(direction, summed, out=np.zeros_like(direction), where=summed != 0))
    for i in range(direction.shape[2]):
        spatial_features.append(direction[:, :, i:i+1].permute(2, 0, 1))
    spatial_view = torch.cat(tuple(spatial_features), 0)
    #print('spatial_view:',spatial_view.shape)

    combine_view = torch.cat([spatial_view, transition_view], 0)
    #print('combine_view:',combine_view.shape)

    combine = combine_view.numpy()
    np.save(img_path.replace('line.png', 'combine.npy').replace('selected', 'spatialtrans'), combine)
    print(img_path.replace('line.png', 'combine.npy').replace('selected', 'spatialtrans'),'stored!')

    #centerline_img = img_transform(centerline_img)
    #region_img = img_transform(region_img)


origin/0_5_combine.npy stored!


In [ ]:
#Observe
combine = combine_view.numpy()
print('combine',combine.shape)
spatial, transition = combine_view.split([11,128],dim=0)
print('spatial:',spatial.shape)
print('transition:',transition.shape)


combine (139, 256, 256)
spatial: torch.Size([11, 256, 256])
transition: torch.Size([128, 256, 256])
